In [3]:
# saas_analysis_full.py
# Full script: load, clean, Top-N, static plots (Matplotlib) + interactive (Plotly)
# Assumes the Excel file is at /mnt/data/Top_100_Saas_Companies_2025.xlsx

import os
import re
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio

In [4]:
# -------------------------
# CONFIG
# -------------------------
INPUT_PATH =  df = pd.read_excel("Top_100_Saas_Companies_2025_Cleaned.xlsx")  # <-- uses uploaded file path
OUTPUT_DIR = r"/mnt/data/saas_analysis_outputs"
TOP_N = 12   # change to 5, 10, 20 as you like
RANDOM_SEED = 42

# ensure output dir exists
os.makedirs(OUTPUT_DIR, exist_ok=True)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)


df.head()

,Company Name,Founded Year,HQ,Industry,Total Funding(USD),Annual Recurring Revenue(USD),Valuation(USD),Employees,Top Investors,Product,Rating
0,Microsoft,1975,"Redmond, WA, USA",Enterprise Software,1.000000e+09,270000000000,3000000000000,221000,"Bill Gates, Paul Allen","Azure, Office 365, Teams",4.4
1,Salesforce,1999,"San Francisco, CA, USA",CRM,6.540000e+07,37900000000,227800000000,75000,"Halsey Minor, Larry Ellison","Sales Cloud, Service Cloud",4.3
2,Adobe,1982,"San Jose, CA, USA",Creative Software,2.500000e+06,19400000000,240000000000,29945,Hambrecht & Quist,"Creative Cloud, Document Cloud",4.5
3,Oracle,1977,"Austin, TX, USA",Database & Enterprise,2.000000e+03,52900000000,350000000000,143000,"Larry Ellison, Bob Miner","Oracle Cloud, NetSuite",4.0
4,SAP,1972,"Walldorf, Germany",Enterprise Software,NaN,32500000000,215000000000,107415,"Dietmar Hopp, Klaus Tschira","S/4HANA, SuccessFactors",4.1


In [ ]:
# Utilities - robust column detection and numeric conversion
# -------------------------

def detect_col(cols, candidates):
    """Return first column name in cols that contains any candidate substring (case-insensitive)."""
    cols_lower = {c.lower(): c for c in cols}
    for cand in candidates:
        for c_lower, c_orig in cols_lower.items():
            if cand.lower() in c_lower:
                return c_orig
    return None

def convert_shorthand_to_number(x):
    """Convert values like '1B', '2.5M', '750K', '120,000', '3,4M' -> numeric (float).
       Handles strings, numbers, and NaN. Returns np.nan for invalid values.
    """
    if pd.isna(x):
        return np.nan
    # If already numeric, return it
    if isinstance(x, (int, float, np.integer, np.floating)) and not isinstance(x, bool):
        return float(x)
    s = str(x).strip()
    if s == "":
        return np.nan
    # remove currency symbols and spaces
    s = s.replace("$", "").replace(" ", "")
    # fix decimal comma to decimal point in cases like 3,4M
    # But be careful with thousands commas: we'll remove commas once we detect suffix or decimal
    # handle patterns like "1,200.75K" -> remove commas first, but "3,4M" should become "3.4M"
    # Strategy: if string has both ',' and '.', assume ',' is thousands separator -> remove commas
    # If string has only ',', assume it's decimal comma -> replace with '.'
    if ',' in s and '.' in s:
        s = s.replace(',', '')
    elif ',' in s and '.' not in s:
        # replace decimal comma with dot
        s = s.replace(',', '.')
    # remove any remaining commas
    s = s.replace(',', '')

    # handle parentheses negative numbers like (1.2M)
    if s.startswith('(') and s.endswith(')'):
        s = '-' + s[1:-1]

    # regex capture numeric part and optional suffix
    m = re.match(r'^([+-]?[0-9]*\.?[0-9]+)([BbMmKk]?)$', s)
    if m:
        num_str = m.group(1)
        suffix = m.group(2).upper()
        try:
            num = float(num_str)
        except:
            return np.nan
        if suffix == 'B':
            return num * 1e9
        elif suffix == 'M':
            return num * 1e6
        elif suffix == 'K':
            return num * 1e3
        else:
            return num
    # As a last resort, attempt to parse as float after stripping non-digit chars
    try:
        cleaned = re.sub(r'[^\d\.\-]', '', s)
        if cleaned == '':
            return np.nan
        return float(cleaned)
    except:
        return np.nan

In [ ]:
# Load the file and detect columns
# -------------------------
print("Loading:", INPUT_PATH)
df = pd.read_excel('Top_100_Saas_Companies_2025_Cleaned.xlsx')  # openpyxl is robust for xlsx

print("Columns found:")
print(df.columns.tolist())

# detect likely column names (list of candidate substrings)
col_company = detect_col(df.columns, ["company", "company_name", "name"])
col_year = detect_col(df.columns, ["founded", "year", "founded_year"])
col_hq = detect_col(df.columns, ["hq", "headquarter", "headquarters", "location"])
col_industry = detect_col(df.columns, ["industry"])
col_funding = detect_col(df.columns, ["funding", "total_funding"])
col_arr = detect_col(df.columns, ["arr", "annual_recurring_revenue", "annual recurring"])
col_valuation = detect_col(df.columns, ["valuation", "value"])
col_employees = detect_col(df.columns, ["employee", "employees", "team size"])
col_investors = detect_col(df.columns, ["investor", "top_investors"])
col_product = detect_col(df.columns, ["product", "products"])
col_rating = detect_col(df.columns, ["rating", "g2", "score"])

print("\nMapped columns:")
print("company:", col_company)
print("founded/year:", col_year)
print("hq:", col_hq)
print("industry:", col_industry)
print("funding:", col_funding)
print("arr:", col_arr)
print("valuation:", col_valuation)
print("employees:", col_employees)
print("investors:", col_investors)
print("product:", col_product)
print("rating:", col_rating)

Loading:     Company Name  Founded Year                      HQ               Industry  \
0      Microsoft          1975        Redmond, WA, USA    Enterprise Software   
1     Salesforce          1999  San Francisco, CA, USA                    CRM   
2          Adobe          1982       San Jose, CA, USA      Creative Software   
3         Oracle          1977         Austin, TX, USA  Database & Enterprise   
4            SAP          1972       Walldorf, Germany    Enterprise Software   
..           ...           ...                     ...                    ...   
95      CircleCI          2011  San Francisco, CA, USA                  CI/CD   
96  ServiceTitan          2012       Glendale, CA, USA          Field Service   
97       Procore          2002    Carpinteria, CA, USA           Construction   
98    Automattic          2005  San Francisco, CA, USA         Web Publishing   
99         Vimeo          2004       New York, NY, USA         Video Platform   

    Total Funding(

In [ ]:
# rating -> convert to numeric safely if exists
if col_rating:
    df[col_rating + "_num"] = df[col_rating].apply(convert_shorthand_to_number)

In [ ]:
# founded year -> make integer year
df['Founded Year'] = pd.to_datetime(df['Founded Year'], format='%Y')

In [ ]:
# founded year -> make integer year
def parse_year(x):
    if pd.isna(x):
        return np.nan
    if isinstance(x, (int, float, np.integer, np.floating)):
        return int(x)
    s = str(x).strip()
    # try YYYY
    m = re.search(r'(\d{4})', s)
    if m:
        return int(m.group(1))
    # try parse datetime
    try:
        t = pd.to_datetime(s, errors='coerce')
        if pd.isna(t):
            return np.nan
        return int(t.year)
    except:
        return np.nan

In [ ]:
print("Parsing founded year...")
df['founded_year_clean'] = df[col_year].apply(parse_year)

Parsing founded year...


In [ ]:
# Create era bins
bins = [0, 1999, 2009, 2019, 3000]
labels = ["Before 2000", "2000–2009", "2010–2019", "2020–Present"]
df['era'] = pd.cut(df['founded_year_clean'], bins=bins, labels=labels)

In [ ]:
column_map = {
    "company": "Company Name",
    "year": "Founded Year",
    "industry": "Industry",
    "funding": "Total Funding(USD)",
    "arr": "ARR(USD)",
    "valuation": "Valuation(USD)",
    "employees": "Employees",
    "rating": "Rating",
}

print(df.columns.to_list())


['Company Name', 'Founded Year', 'HQ', 'Industry', 'Total Funding(USD)', 'Annual Recurring Revenue(USD)', 'Valuation(USD)', 'Employees', 'Top Investors', 'Product', 'Rating', 'Rating_num', 'founded_year_clean', 'era']


In [ ]:
# drop rows with no company name
df = df[df[col_company].notna()].copy()

In [ ]:
# save cleaned CSV for inspection
clean_csv_path = os.path.join(OUTPUT_DIR, "saas_data_cleaned.csv")
df.to_csv(clean_csv_path, index=False)
print("Cleaned data saved to:", clean_csv_path)

Cleaned data saved to: /mnt/data/saas_analysis_outputs\saas_data_cleaned.csv


In [ ]:
# -------------------------
# Helper: Top N selection
# -------------------------

def top_n_by(df_in, metric_col, n=TOP_N):
    return df_in.sort_values(metric_col, ascending=False).head(n)

In [ ]:
# -------------------------
# PLOTTING: Matplotlib static charts
# -------------------------
import seaborn as sns

plt.style.use('ggplot')  # minimal, clean
def save_fig(fig, filename):
    path = os.path.join(OUTPUT_DIR, filename)
    fig.savefig(path, dpi=220, bbox_inches='tight')
    print("Saved:", path)

In [ ]:
# 1) Top N Industries by Average ARR
fig, ax = plt.subplots(figsize=(10,6))
industry_avgs = df.groupby(col_industry)['Annual Recurring Revenue(USD)'].mean().dropna().sort_values(ascending=False).head(TOP_N)
industry_avgs[::-1].plot(kind='barh', ax=ax)  # reverse for top-down
ax.set_xlabel("Average ARR (USD)")
ax.set_title(f"Top {TOP_N} Industries by Average ARR")
save_fig(fig, f"top_{TOP_N}_industries_avg_arr.png")
plt.close(fig)

Saved: /mnt/data/saas_analysis_outputs\top_12_industries_avg_arr.png


In [ ]:
# 2) Top N Companies: ARR vs Funding (Matplotlib scatter with log scales + jitter)
top_by_arr = top_n_by(df, 'Annual Recurring Revenue(USD)', TOP_N)
fig, ax = plt.subplots(figsize=(10,6))
x = top_by_arr['Total Funding(USD)']
y = top_by_arr['Annual Recurring Revenue(USD)']
# add slight jitter so labels don't overlap
jitter_x = x * (1 + (np.random.rand(len(x)) - 0.5) * 0.01)
jitter_y = y * (1 + (np.random.rand(len(y)) - 0.5) * 0.01)
sizes = (top_by_arr['Valuation(USD)'].fillna(0) / (1e7)).clip(lower=10, upper=200)
sc = ax.scatter(jitter_x, jitter_y, s=sizes, alpha=0.7)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel("Total Funding (USD) - log scale")
ax.set_ylabel("ARR (USD) - log scale")
ax.set_title(f"Top {TOP_N} Companies by ARR: ARR vs Funding")
# annotate points with abbreviated names
for i, (xi, yi, lbl) in enumerate(zip(jitter_x, jitter_y, top_by_arr[col_company])):
    ax.annotate(lbl, (xi, yi), textcoords="offset points", xytext=(3,3), fontsize=8, alpha=0.9)
save_fig(fig, f"top_{TOP_N}_arr_vs_funding.png")
plt.close(fig)

Saved: /mnt/data/saas_analysis_outputs\top_12_arr_vs_funding.png


In [ ]:
# 3) Top N Companies by ARR per $1 funding (Efficiency)
top_eff = top_n_by(df.dropna(subset=['arr_per_dollar']), 'arr_per_dollar', TOP_N)
fig, ax = plt.subplots(figsize=(10,6))
ax.barh(top_eff[col_company], top_eff['arr_per_dollar'])
ax.invert_yaxis()
ax.set_xlabel("ARR per $1 of funding")
ax.set_title(f"Top {TOP_N} Companies by ARR per $1 Funding")
save_fig(fig, f"top_{TOP_N}_arr_per_dollar.png")
plt.close(fig)


Saved: /mnt/data/saas_analysis_outputs\top_12_arr_per_dollar.png


In [ ]:
# 4) Top N Companies by ARR (bar)
top_arr = top_n_by(df, 'Annual Recurring Revenue(USD)', TOP_N)
fig, ax = plt.subplots(figsize=(12,6))
ax.bar(top_arr[col_company], top_arr['Annual Recurring Revenue(USD)'])
plt.xticks(rotation=45, ha='right')
ax.set_ylabel("ARR (USD)")
ax.set_title(f"Top {TOP_N} Companies by ARR")
save_fig(fig, f"top_{TOP_N}_companies_by_arr.png")
plt.close(fig)

Saved: /mnt/data/saas_analysis_outputs\top_12_companies_by_arr.png


In [ ]:
# 5) Top N Companies: Employees vs ARR (bubble)
top_emp = top_n_by(df, 'Annual Recurring Revenue(USD)', TOP_N)
fig, ax = plt.subplots(figsize=(10,6))
sx = top_emp['Employees'].replace(0, np.nan).fillna(1)
sy = top_emp['Annual Recurring Revenue(USD)']
ss = (top_emp['Valuation(USD)'].fillna(0) / 1e7).clip(lower=20, upper=300)
ax.scatter(sx, sy, s=ss, alpha=0.6)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel("Employees (log scale)")
ax.set_ylabel("ARR (log scale)")
ax.set_title(f"Top {TOP_N} Companies — Employees vs ARR (Bubble=Valuation)")
for i, (xi, yi, lbl) in enumerate(zip(sx, sy, top_emp[col_company])):
    ax.annotate(lbl, (xi, yi), textcoords="offset points", xytext=(3,3), fontsize=8)
save_fig(fig, f"top_{TOP_N}_employees_vs_arr.png")
plt.close(fig)

Saved: /mnt/data/saas_analysis_outputs\top_12_employees_vs_arr.png


In [ ]:
# 6) Era counts (Top N eras probably <=4 but keep Top_N support)
era_counts = df['era'].value_counts().sort_values(ascending=False).head(TOP_N)
fig, ax = plt.subplots(figsize=(8,5))
era_counts.plot(kind='bar', ax=ax)
ax.set_ylabel("Number of Companies")
ax.set_title("Companies by Founding Era")
save_fig(fig, f"companies_by_era.png")
plt.close(fig)

Saved: /mnt/data/saas_analysis_outputs\companies_by_era.png


In [ ]:
# 7) Ratings distribution (histogram) - if rating exists
if col_rating:
    fig, ax = plt.subplots(figsize=(8,5))
    ax.hist(df['Rating'].dropna(), bins=20, edgecolor='black', alpha=0.7)
    ax.set_xlabel("Rating")
    ax.set_ylabel("Number of Companies")
    ax.set_title("Distribution of Company Ratings")
    save_fig(fig, "ratings_distribution.png")
    plt.close(fig)

Saved: /mnt/data/saas_analysis_outputs\ratings_distribution.png


In [ ]:
# -------------------------
# Interactive Charts (Plotly)
# -------------------------
pio.templates.default = "plotly_white"

In [ ]:
# A) Interactive ARR vs Funding scatter (Top_N)
fig = px.scatter(
    top_by_arr,
    x='Total Funding(USD)',
    y='Annual Recurring Revenue(USD)',
    size='Valuation(USD)',
    color=col_industry,
    hover_name=col_company,
    log_x=True,
    log_y=True,
    title=f"Top {TOP_N} Companies: ARR vs Funding (Interactive)"
)
html_path = os.path.join(OUTPUT_DIR, f"interactive_top_{TOP_N}_arr_vs_funding.html")
fig.write_html(html_path)
print("Saved interactive plot:", html_path)

Saved interactive plot: /mnt/data/saas_analysis_outputs\interactive_top_12_arr_vs_funding.html


In [ ]:
# B) Interactive Funding Efficiency (Top_N)
fig = px.scatter(
    top_eff,
    x='Total Funding(USD)',
    y='arr_per_dollar',
    size='Valuation(USD)',
    color=col_industry,
    hover_name=col_company,
    log_x=True,
    title=f"Top {TOP_N} Companies: ARR per $1 Funding (Interactive)"
)
html_path = os.path.join(OUTPUT_DIR, f"interactive_top_{TOP_N}_arr_per_dollar.html")
fig.write_html(html_path)
print("Saved interactive plot:", html_path)

Saved interactive plot: /mnt/data/saas_analysis_outputs\interactive_top_12_arr_per_dollar.html


In [ ]:
# C) Interactive Employees vs ARR (Top_N)
fig = px.scatter(
    top_emp,
    x='Employees',
    y='Annual Recurring Revenue(USD)',
    size='Valuation(USD)',
    color=col_industry,
    hover_name=col_company,
    log_x=True,
    log_y=True,
    title=f"Top {TOP_N} Companies: Employees vs ARR (Interactive)"
)
html_path = os.path.join(OUTPUT_DIR, f"interactive_top_{TOP_N}_employees_vs_arr.html")
fig.write_html(html_path)
print("Saved interactive plot:", html_path)

Saved interactive plot: /mnt/data/saas_analysis_outputs\interactive_top_12_employees_vs_arr.html


In [ ]:
# D) Industry average ARR bar (Top_N)
industry_avg_df = df.groupby(col_industry)['Annual Recurring Revenue(USD)'].mean().reset_index().sort_values('Annual Recurring Revenue(USD)', ascending=False).head(TOP_N)
fig = px.bar(industry_avg_df[::-1], x='Annual Recurring Revenue(USD)', y=col_industry, orientation='h', title=f"Top {TOP_N} Industries by Average ARR (Interactive)")
html_path = os.path.join(OUTPUT_DIR, f"interactive_top_{TOP_N}_industries_avg_arr.html")
fig.write_html(html_path)
print("Saved interactive plot:", html_path)

Saved interactive plot: /mnt/data/saas_analysis_outputs\interactive_top_12_industries_avg_arr.html


In [ ]:
# E) Ratings distribution (interactive)
if col_rating:
    fig = px.histogram(df, x='Rating', nbins=20, title="Ratings Distribution (Interactive)")
    html_path = os.path.join(OUTPUT_DIR, "interactive_ratings_distribution.html")
    fig.write_html(html_path)
    print("Saved interactive plot:", html_path)

Saved interactive plot: /mnt/data/saas_analysis_outputs\interactive_ratings_distribution.html


In [ ]:
# Save a short JSON summary of key stats
summary = {
    "total_companies": int(df.shape[0]),
    "avg_rating": float(df['Rating'].mean()) if col_rating else None,
    "median_arr": float(df['Annual Recurring Revenue(USD)'].median()),
    "median_funding": float(df['Total Funding(USD)'].median()),
    "median_valuation": float(df['Valuation(USD)'].median()),
}
import json
with open(os.path.join(OUTPUT_DIR, "summary_stats.json"), "w") as f:
    json.dump(summary, f, indent=2)
print("Saved summary stats JSON.")

print("\nALL DONE. Outputs written to:", OUTPUT_DIR)
print("Static PNGs:")
for fn in os.listdir(OUTPUT_DIR):
    if fn.lower().endswith(".png"):
        print(" -", os.path.join(OUTPUT_DIR, fn))
print("Interactive HTML files:")
for fn in os.listdir(OUTPUT_DIR):
    if fn.lower().endswith(".html"):
        print(" -", os.path.join(OUTPUT_DIR, fn))

Saved summary stats JSON.

ALL DONE. Outputs written to: /mnt/data/saas_analysis_outputs
Static PNGs:
 - /mnt/data/saas_analysis_outputs\companies_by_era.png
 - /mnt/data/saas_analysis_outputs\ratings_distribution.png
 - /mnt/data/saas_analysis_outputs\top_12_arr_per_dollar.png
 - /mnt/data/saas_analysis_outputs\top_12_arr_vs_funding.png
 - /mnt/data/saas_analysis_outputs\top_12_companies_by_arr.png
 - /mnt/data/saas_analysis_outputs\top_12_employees_vs_arr.png
 - /mnt/data/saas_analysis_outputs\top_12_industries_avg_arr.png
Interactive HTML files:
 - /mnt/data/saas_analysis_outputs\interactive_ratings_distribution.html
 - /mnt/data/saas_analysis_outputs\interactive_top_12_arr_per_dollar.html
 - /mnt/data/saas_analysis_outputs\interactive_top_12_arr_vs_funding.html
 - /mnt/data/saas_analysis_outputs\interactive_top_12_employees_vs_arr.html
 - /mnt/data/saas_analysis_outputs\interactive_top_12_industries_avg_arr.html


In [ ]:
import os
print(os.getcwd())

C:\Users\sibusiso\Downloads\Top 100 SaaS CompaniesStartups 2025


In [ ]:
%matplotlib inline